<a href="https://colab.research.google.com/github/pfmoro/Black-Scholes-Greeks/blob/master/Ion_Cannon_Operational.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ion Cannon é uma automação para análise de opções sobre ações no mercado americano.

Possui capacidades de cálculo dos principais indicadores tecnicos de mercado, bem como análise de gregas de opções

In [0]:
#Parametros De inicialização:
import datetime
from datetime import date

OptionDate="2019-08-16"
Asset="SPY"
WindowVol=30
WindowTrade=5
Quantity=1


if OptionDate =="":
  fromDate=(date.today()+datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  toDate=(date.today()+datetime.timedelta(days=15)).strftime('%Y-%m-%d')
else:
  fromDate=OptionDate
  toDate=(datetime.datetime.strptime(OptionDate,'%Y-%m-%d')+datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  


StartDate='01/01/2016'
EndDate= date.today().strftime('%d/%m/%Y') #'15/06/2016'

In [0]:
pip install scipy==1.1  #correção do erro do scipy 1.3


     |████████████████████████████████| 31.2MB 1.4MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.3.0
    Uninstalling scipy-1.3.0:
      Successfully uninstalled scipy-1.3.0


In [0]:
pip install tdameritrade

  Created wheel for tdameritrade: filename=tdameritrade-0.0.8-py2.py3-none-any.whl size=5425 sha256=115adcce0cf03096c557967a042dc392fb9f71839362d0efe384f19c4f9138de
  Stored in directory: /root/.cache/pip/wheels/b9/85/82/5dacd1a772d57d1368476a2b222a9dbb611cfd2354c30f5d25
Successfully built tdameritrade


In [0]:
#@title
#Import requirements:
import csv
import json
import urllib.request
import tempfile
import time
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
#from tdameritrade import TDClient

from scipy.stats import norm
from scipy.stats import gaussian_kde
from scipy.stats import powerlaw
from scipy.stats import ks_2samp
from scipy.integrate import trapz
import scipy.stats
import math
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint
import numpy as np
import holidays

import pandas
import requests
from pandas.io.json import json_normalize

__all__ = ['getHistoricalData','getInstrumentInformation']

In [0]:
#@title  { vertical-output: true }
#Functions:
def getFundamentals(symbol):
    """
    Returns a dataframe of the stock fundamental information.
    >>> if(True): print(1)
    1
    """

    if type(symbol) is not str:
        return 'Symbol is not correct format'
    searchUrl =  "https://api.tdameritrade.com/v1/instruments?apikey=0JZUVH29AHKZJ&symbol=%s&projection=fundamental" % symbol
    headers = 'Authorization'
    req = urllib.request.Request(searchUrl)
    req.add_header(headers, '')
    r = urllib.request.urlopen(req)
    js = json.loads(r.read().decode('utf-8'))
    data = json_normalize(js,symbol)
    return data

def getFullFundamentals(symbol):
    """
    Returns a dataframe of the stock fundamental information.
    >>> if(True): print(1)
    1
    """

    if type(symbol) is not str:
        return 'Symbol is not correct format'
    searchUrl =  "https://api.tdameritrade.com/v1/instruments?apikey=0JZUVH29AHKZJ&symbol=%s&projection=fundamental" % symbol
    headers = 'Authorization'
    req = urllib.request.Request(searchUrl)
    req.add_header(headers, '')
    r = urllib.request.urlopen(req)
    js = json.loads(r.read().decode('utf-8'))
    data = js
    #data = json_normalize(js,symbol)
    return data

def getMorningstar(sym='',type='',columnYear='5',period='12'):
  flag=True
  while flag:
    try:
    #period: 3 for quarterly, 12 for annual
      balanceSheetUrl = "http://financials.morningstar.com/ajax/ReportProcess4CSV.html?" + \
          "t=" + sym + \
          "&reportType=" + type + \
          "&period=" + period + \
          "&dataType=A&order=asc&columnYear=" + columnYear + "&number=3"



      with urllib.request.urlopen(balanceSheetUrl) as r:
          data = r.read().decode('utf-8')

          csv_output = {}
          for row in csv.reader(data.splitlines()):
              if len(row) > 1:
                  if len(csv_output) == 0:
                      csv_output['index'] = row[1:]
                  else:
                      csv_output[row[0]] = row[1:]
          exit=pandas.DataFrame(csv_output, index=csv_output['index'])
          flag=False
    except:
          flag=True
  return exit

def getIntradayData(symbol, d):
    """
    Function to get the historical quotes and splits for a symbot.
    Must be provided with a symbol(string) and a date in the format Year-Month-Date
    e.g. 2019-02-01 Which is the 2nd February 2019.
    Returns a dataframe.
    """
    if type(symbol) is not str:
        return 'Symbol is not correct format'
    pattern = '%Y-%m-%d'
    epoch = int(time.mktime(time.strptime(d,pattern)))*1000 #TD exige Unix timestamp em milisegundos
    endtime=epoch+82800000 #timestamp do final do dia, url ajustada para dados diarios, até um mes antes da data base
    searchUrl =  "https://api.tdameritrade.com/v1/marketdata/%s/pricehistory?apikey=0JZUVH29AHKZJ&periodType=day&period=1&frequencyType=minute&frequency=1&endDate=%i" % (symbol, endtime )
    headers = 'Authorization'
    req = urllib.request.Request(searchUrl)
    req.add_header(headers, '')
    r = urllib.request.urlopen(req)
    js = json.loads(r.read().decode('utf-8'))
    data = json_normalize(js, 'candles')
    data['datetime'].astype(float)
    #data['datetime']=data['datetime']/1000
    data['datetime']=pandas.to_datetime(data['datetime'], unit='ms')
    return data

def getHistoricalData(symbol, d):
    """
    Function to get the historical quotes and splits for a symbot.
    Must be provided with a symbol(string) and a date in the format Year-Month-Date
    e.g. 2019-02-01 Which is the 2nd February 2019.
    Returns a dataframe.
    """
    if type(symbol) is not str:
        return 'Symbol is not correct format'
    pattern = '%Y-%m-%d'
    epoch = int(time.mktime(time.strptime(d,pattern)))*1000 #TD exige Unix timestamp em milisegundos
    endtime=epoch+82800000 #timestamp do final do dia, url ajustada para dados diarios, até um mes antes da data base
    searchUrl =  "https://api.tdameritrade.com/v1/marketdata/%s/pricehistory?apikey=0JZUVH29AHKZJ&periodType=month&frequencyType=daily&startDate=%i&endDate=%i" % (symbol, epoch, endtime )
    headers = 'Authorization'
    req = urllib.request.Request(searchUrl)
    req.add_header(headers, '')
    r = urllib.request.urlopen(req)
    js = json.loads(r.read().decode('utf-8'))
    data = json_normalize(js, 'candles')
    return data

def getOptionChain(symbol, fromDate, toDate, strikeCount='10'):
    if type(symbol) is not str:
        return 'Symbol is not correct format - str'
    if type(strikeCount) is not str:
      return 'strikeCount is not correct format - str'
    if type(fromDate) is not str:
      return 'fromDate is not correct format - str'
    if type(toDate) is not str:
      return 'toDate is not correct format - str'
    ret = []
    searchUrl="https://api.tdameritrade.com/v1/marketdata/chains?apikey=0JZUVH29AHKZJ&symbol=" + symbol + "&contractType=ALL&strikeCount=" + strikeCount + "&includeQuotes=TRUE&strategy=SINGLE&range=ALL&fromDate="+ fromDate +"&toDate=" + toDate + "&expMonth=ALL&optionType=ALL"
    headers = 'Authorization'
    req = urllib.request.Request(searchUrl)
    req.add_header(headers, '')
    r = urllib.request.urlopen(req)
    dat  = json.loads(r.read().decode('utf-8'))
    for date in dat['callExpDateMap']:
        for strike in dat['callExpDateMap'][date]:
                ret.extend(dat['callExpDateMap'][date][strike])
    for date in dat['putExpDateMap']:
        for strike in dat['putExpDateMap'][date]:
                ret.extend(dat['putExpDateMap'][date][strike])

    df = pd.DataFrame(ret)
    for col in ('tradeTimeInLong', 'quoteTimeInLong', 'expirationDate', 'lastTradingDay'):
            df[col] = pd.to_datetime(df[col], unit='ms')
    return df
  
def getInstrumentInformation(symbol):
    """
    Function to return the fundamental information for a ticker.
    Requires symbol as a string.
    Returns a dictionary with the symbol being the top level and then the fundamentals, incomeStatement
    cashflow and balancesheet being listed under those keys.
    Each key contains a DataFrame.
    """
    f = getFundamentals(symbol)
    incomeStatement, cf, bs = (getMorningstar(sym=symbol,type=x) for x in ['is','cf','bs'])
    print ("Done processing")
    return {
        symbol:{
            'fundamentals':f,
            'incomeStatement':incomeStatement,
            'cashflow':cf,
            'balancesheet':bs
    }}
  
def getBalanceSheet(symbol):
    #receives a Ticker and returns a dataframe containing its balance sheet
    mydata = getInstrumentInformation(symbol)
    bs=mydata[symbol]['balancesheet']
    #list(my_dataframe.columns.values)
    return bs
  
def ListColumns(my_dataframe):
   #lists columns of a dataframe:
   mylist=[]
   list(my_dataframe.columns.values)
   mylist= list(my_dataframe.columns.values)
   return mylist

def getCashflow(symbol):
    #receives a Ticker and returns a dataframe containing its Cashflow
    mydata = getInstrumentInformation(symbol)
    bs=mydata[symbol]['cashflow']
    #list(my_dataframe.columns.values)
    return bs
  
def getIncomeStatement(symbol):
    #receives a Ticker and returns a dataframe containing its income statement
    mydata = getInstrumentInformation(symbol)
    bs=mydata[symbol]['incomeStatement']
    #list(my_dataframe.columns.values)
    return bs  

def linearRegressionCoefs(X,Y):
  #receives X and Y vectors and returns regression coeficients
  X = sm.add_constant(X)
  result=sm.OLS(Y,X).fit()
  myCoef=result.params #returns list in format [intercept(alpha), slope(beta)]
  return myCoef

def getDailyQuotes(Asset,date):	
  #Receives Asset and dat on aaaa-mm-dd format and returns a pandas dataframe of a single line, containing high,low,close,open and volume for the date
  myquotes=getHistoricalData(Asset,date)
  myquotes['high'].astype(np.float).iloc[-1]
  myquotes["close"].astype(np.float).iloc[-1]
  myquotes["low"].astype(np.float).iloc[-1]
  myquotes["open"].astype(np.float).iloc[-1]
  myquotes["volume"].astype(np.float).iloc[-1]
  

  mymax=myquotes["high"].iloc[-1]
  mymin=myquotes["low"].iloc[-1]  
  myvolume=myquotes["volume"].iloc[-1]
  myclose=myquotes["close"].iloc[-1]
  myopen=myquotes["open"].iloc[-1]

  dayframe=pandas.DataFrame({'date': [date] ,'close': [myclose], 'open': [myopen],'high': [mymax],'low': [mymin],'volume': [myvolume]}, columns=['date', 'close', 'open', 'high','low','volume'])

  return dayframe


Código abaixo contem funções fundamentalistas de ações:

MoroScoreCalculator(Asset)
Zscore() - TODO

In [0]:
#@title
#Fundamentals Analysis: Z-Score and MoroScore Calculation


def MoroScoreCalculator(asset):
  f = getFullFundamentals(asset)
  assettype=f[asset]['description']
  print('evaluating: ' + str(assettype))
  #moro score
  moroscore=0
  tempmoroscore=0
  #fluxo de caixa operacional positivo nos ultimos 5 anos
  CF=getCashflow(asset)
  BS=getBalanceSheet(asset)
  IS=getIncomeStatement(asset)
  FD=getFundamentals(asset)
  for i in range(5):
    if float(CF['Operating cash flow'][i])>0:
      print("Operating Cash Flow year " + str(i+1) +" OK")
      tempmoroscore=1
    else:
      print("Operating Cash Flow year " + str(i+1) +" NOK")
      moroscore=0
      tempmoroscore=0
  moroscore=moroscore+tempmoroscore
  #Relacao divida ativo maior que 50%:
  if (float(BS['Short-term debt'][4])+float(BS['Long-term debt'][4]))/float(BS['Total assets'][4])>0.5 and moroscore!=0:
    moroscore=0
    tempmoroscore=0
  else:
    tempmoroscore=1
  
  if (float(BS['Short-term debt'][4])+float(BS['Long-term debt'][4]))/float(BS['Total assets'][4])>0.5:
    print("Debt to Asset Ratio NOK")
  else:
    print("Debt to Asset Ratio OK")
  moroscore=moroscore+tempmoroscore

  #NOPLAT cobre pelo menos duas vezes as despesas com juros:
  if (float(IS['Net income'][4])+float(IS['Provision for income taxes'][4]))> 2*float(IS['Interest Expense'][4]) and moroscore!=0:
    tempmoroscore=1
  else:
    moroscore=0
    tempmoroscore=0
  
  if (float(IS['Net income'][4])+float(IS['Provision for income taxes'][4]))> 2*float(IS['Interest Expense'][4]):
    print("OK NOPLAT")
  else:
    print("NOK NOPLAT")

  moroscore=moroscore+tempmoroscore

  #Avaliar se receitas, lucro ou margem de lucro(lucro/receitas) sao crescentes:
  parameters=[]
  #Receitas:
  initmoroscore=moroscore
  X=list(range(5))
  Y=list(map(float, IS['Revenue']))
  Y=Y[:5]
  parameters=linearRegressionCoefs(X,Y)
  if parameters[1]<0 or moroscore ==0:
    moroscore=0
    tempmoroscore=0
  else:
    tempmoroscore=1
  
  if parameters[1]<0:
    print("Revenue in downward Trend")
  else:
    print("Revenue in upward Trend")

  moroscore = moroscore+tempmoroscore
  #lucro:
  Y=list(map(float, IS['Net income available to common shareholders']))
  Y=Y[:5]
  parameteres=linearRegressionCoefs(X,Y)
  if parameters[1]<0 or moroscore==0:
    moroscore=0
    tempmoroscore=0
  else:
    tempmoroscore=1
    
  if parameters[1]<0:
    print("Profit in downward Trend")
  else:
    print("Profit in upward Trend")
  moroscore = moroscore+tempmoroscore	

  #margem de lucro:
  Rev=list(map(float, IS['Revenue']))
  Inc=list(map(float, IS['Net income available to common shareholders']))

  Y=(np.array(Rev)/np.array(Inc))
  Y=Y[:5]
  parameters=linearRegressionCoefs(X,Y)
  if parameters[1]<0 or moroscore==0:
    moroscore=0
    tempmoroscore=0
  else:
    tempmoroscore=1
    
  if parameters[1]<0:
    print("Profit margin in downward Trend")
  else:
    print("Profit margin in upward Trend")

  moroscore=moroscore+tempmoroscore
  # teste se a ação está com sobrepreco
  if float(f[asset]['peRatio']) > 30 or mororoscore== 0:
    moroscore=0
    tempmoroscore=0
  else:
    tempmoroscore=1
    
  if float(f[asset]['peRatio']) > 30:
    print('Asset is overpriced')
  else:
    print('Asset is not overpriced')
    
  if float(f[asset]['peRatio']) < 10:
    print('Asset is  discounted')
    tempmoroscore=1
  else:
    print('Asset is not discounted')  
    tempmoroscore=0
  moroscore=moroscore+tempmoroscore 
  
  #price x book:
  if float(f[asset]['pbRatio']) < 2 and mororoscore != 0:  
    tempmoroscore=1
  else:
    tempmoroscore=0
    
  
    
  if float(f[asset]['pbRatio']) < 2:
    print('Asset is book discounted')
    tempmoroscore=1
  else:
    print('Asset is not book discounted')  
    tempmoroscore=0
  moroscore=moroscore+tempmoroscore
  
  #teste se uma das condicoes de receitas, lucro ou margem de lucro sao crescentes
  if initmoroscore==moroscore:
    moroscore=0
  
  #teste se a performance do ativo é melhor que a do benchmark
  
  benchmark="SPY" #ETF que replica o índice S&P 500
  now = datetime.datetime.now()
  #analise de 5 anos:
  End=now.year
  Init=now.year-5
  benchmarkInit=getDailyQuotes(benchmark,str(Init)+"-01-01")
  benchmarkEnd=getDailyQuotes(benchmark,str(End)+"-01-01")
  benchmarkreturn=float(benchmarkEnd["close"][0])/float(benchmarkInit["close"][0])-1
  assetInit=getDailyQuotes(asset,str(Init)+"-01-01")
  AssetEnd=getDailyQuotes(asset,str(End)+"-01-01")
  assetreturn=float(AssetEnd["close"][0])/float(assetInit["close"][0])-1 
  #analise de 1 ano:
  if assetreturn < benchmarkreturn:
    beatbenchmark5years=False
  else:
    beatbenchmark5years=True
  End=now.year
  Init=now.year-1
  benchmarkInit=getDailyQuotes(benchmark,str(Init)+"-01-01")
  benchmarkEnd=getDailyQuotes(benchmark,str(End)+"-01-01")
  benchmarkreturn=float(benchmarkEnd["close"][0])/float(benchmarkInit["close"][0])-1
  assetInit=getDailyQuotes(asset,str(Init)+"-01-01")
  AssetEnd=getDailyQuotes(asset,str(End)+"-01-01")
  assetreturn=float(AssetEnd["close"][0])/float(assetInit["close"][0])-1
  if assetreturn < benchmarkreturn:
    beatbenchmarklastyear=False
  else:
    beatbenchmarklastyear=True  
   
  if moroscore !=0:
    if beatbenchmarklastyear or beatbenchmark5years:
     moroscore=moroscore+1
    else:
     moroscore=0

  if beatbenchmark5years:
   print("Asset performed better than benchmark in the last 5 years")
  else:
   print("Asset did not perform better than benchmark in the past 5 years")
  
  if beatbenchmarklastyear:
   print("Asset performed better than benchmark in the last year")
  else:
   print("Asset did not perform better than benchmark in the last year")
  print("Moro's Score: "+ str(moroscore))
  
  return moroscore
  
  






  

Código abaixo contém funções:

média móvel exponencial: ExpMovingAverageArray(prices, lengthOfMA)

Oscilador: StochasticOscilatorArray(prices,lenghtOfOscilator)

MACD: MACD_Array(prices, lengthOfMALong, lengthOfMAShort)

Volatilidade(Amplitude percentual de preços): Volatility(Minprice,Maxprice,Closeprice)

In [0]:
#@title
#Financial Technical Functons:

def ExpMovingAverageArray(pricesInput, lengthOfMA):
    prices =np.array(pricesInput)
    cont=0
    EMAArray=[]
    np.nan_to_num(prices)
    currentMovingAverage=0
    while cont <len(prices):
        if cont< lengthOfMA:
         currentMovingAverage = np.average(prices[0:cont])
        else:
         currentMovingAverage = (float(prices[cont])  - float(currentMovingAverage)) * (2/(float(lengthOfMA) +1)) + float(currentMovingAverage)
        EMAArray.append(currentMovingAverage)
        cont=cont+1        
    return EMAArray

def StochasticOscilatorArray(prices,lenghtOfOscilator):
    cont=0
    StoArray=[]
    while cont <len(prices):
        if (cont > lenghtOfOscilator):
            minimus=min(float(prices[cont]),float(min(prices[(cont-lenghtOfOscilator):cont])))
            maximus=max(float(prices[cont]),float(max(prices[(cont-lenghtOfOscilator):cont])))
            Oscilator = ((float(prices[cont]) - minimus)/(maximus - minimus))
        else:
            Oscilator = 0.5
        cont=cont+1
        StoArray.append(Oscilator)
    return StoArray   

def MACD_Array(prices, lengthOfMALong, lengthOfMAShort, lengthOfMASignal):
  EMALong = pd.DataFrame(ExpMovingAverageArray(prices, lengthOfMALong))
  EMAShort = pd.DataFrame(ExpMovingAverageArray(prices, lengthOfMAShort))
  #EMASignal = ExpMovingAverageArray(prices, lengthOfMAShort)
  #where_are_NaNs = isnan(EMALong)
  #EMALong[where_are_NaNs] = 0
  #where_are_NaNs = isnan(EMAShort)
  #EMAShort[where_are_NaNs] = 0
  #MACD= np.subtract(EMALong, EMAShort)
  MACDint=list(EMALong.sub(EMAShort, fill_value=0))
  MACD =pd.DataFrame(ExpMovingAverageArray(MACDint, lengthOfMASignal))
  return MACD

def Volatility(Minprice,Maxprice,Closeprice):
  #volatilidade sob conceito de amplitude diária de preço
    vol=[]
    cont=0
    while cont < min(len(Minprice),len(Maxprice),len(Closeprice)):
        vol.append((float(Maxprice[cont]-Minprice[cont])/Closeprice[cont]))
        cont=cont+1
    return vol

Cálculo das Gregas:

S0= Stock Price
X= Strike Price
r=Risk-free Rate
Sigma=Volatility
T=Time to expiration
x=variable

def D1(S0, X, r, Sigma, T):

def D2(S0, X, r, Sigma, T):

Delta(S0, X, r, Sigma, T,type="Call"):

Gamma(S0, X, r, Sigma, T,type="Call"):

Theta(S0, X, r, Sigma, T,type="Call"):

BS(S0, X, r, Sigma, T,type="Call")):

In [0]:
#@title
#THE GREEKS:

#THIS CODE WILL CALCULATE THE BLACK-SCHOLES GREEKS AND OPTIONS PRICE

#S0= Stock Price
#X= Strike Price
#r=Risk-free Rate
#Sigma=Volatility
#T=Time to expiration IN YEARS
#x=variable
def D1(S0, X, r, Sigma, T):
  #a variable that is usedin all calculations
  d1=math.log((S0/X)+(r+(Sigma^2)/2)*T/(Sigma*math.sqrt(T)))
  return d1

def D2(S0, X, r, Sigma, T):
  #a variable that is usedin all calculations
  d2=math.log((S0/X)+(r-(Sigma^2)/2)*T/(Sigma*sqrt(T)))
  return d2

def Nline(x):
 #Derivative of normal cumulative distribution function
 NDer=(1/math.sqrt(2*pi()))*exp(-(x^2)/2)
 return NDer

def Delta(S0, X, r, Sigma, T,type="Call"):
  #variation of option price per unit of variation of stock price
  d1=D1(S0, X, r, Sigma, T)
  if type=="Call":
    myDelta=norm.cdf(d1)
  else:
    myDelta=norm.cdf(d1)-1
  return myDelta

def Gamma(S0, X, r, Sigma, T,type="Call"):
   #variation of option price per unit of variation of delta
  d1=D1(S0, X, r, Sigma, T)
  NL=Nline(d1)
  
  myGamma=NL/(S0*Sigma*math.sqrt(T))  
  return myGamma

def Theta(S0, X, r, Sigma, T,type="Call"):
  #variation of option price per unit of time variation
  d1=D1(S0, X, r, Sigma, T)
  d2=D2(S0, X, r, Sigma, T)
  NL=Nline(d1)
  
  myTheta=S0*NL*Sigma -r*X*exp(-r*T)*norm.cdf(d2)
  return myTheta

def BS(S0, X, r, Sigma, T,type="Call"):
  #returns option price
  d1=D1(S0, X, r, Sigma, T)
  d2=D2(S0, X, r, Sigma, T)
  PV=X*exp(-r*(T))
  if type=="Call":
    price=norm.cdf(d1)*S0-norm.cdf(d2)*PV
  else:
    price=PV-S0+norm.cdf(d1)*S0-norm.cdf(d2)*PV
  return price

Contem as funcoes FullStockDataFrame e FilterDataFrame que montam o data frame e filtram por condicoes especificas, respectivamente.

In [0]:
#@title
#Analytics: this code will retrieve a pandas dataframe containing the prices for further strategy evaluation:
import pandas as pd
from numpy import isnan



def FullStockDataFrame(StartDate, EndDate, Asset, WindowVol=30, WindowTrade=5, verbose = False):
  StartDate=datetime.datetime.strptime(StartDate, '%d/%m/%Y')
  EndDate=datetime.datetime.strptime(EndDate, '%d/%m/%Y')

  ny_holidays=holidays.CountryHoliday("US",prov=None,state="NY")
  StockData=pd.DataFrame()
  CurrentData=pd.DataFrame()

  CurrentDate=StartDate

  while CurrentDate <= EndDate:
    if CurrentDate not in ny_holidays:
      try: 
        CurrentData=getDailyQuotes(Asset,CurrentDate.strftime('%Y-%m-%d'))#checar getDailyQuotes: pega sempre o mesmo valor
        StockData=pd.concat([StockData,CurrentData],ignore_index=True)
      except:
        if verbose == True: print("Unable to fetch data for: "+str(CurrentDate.strftime('%Y-%m-%d')))
    CurrentDate=CurrentDate+datetime.timedelta(days=1)


  StockData["Daily_Log_Return"]=pd.Series(np.log(StockData["close"]/StockData["close"].shift(1)))  
  StockData["Daily_Return"]=(1-StockData["close"]/StockData["close"].shift(1))
  StockData["Hist_Volatility_"+str(WindowVol)+"_Days"]=StockData["Daily_Log_Return"].rolling(window=WindowVol, min_periods=0).std()*1/np.sqrt(WindowVol)
  where_Vol_is_NaNs = np.isnan(StockData["Hist_Volatility_"+str(WindowVol)+"_Days"])
  StockData["Hist_Volatility_"+str(WindowVol)+"_Days"][where_Vol_is_NaNs] = np.log(StockData["close"]/StockData["open"])
  StockData["Next_"+str(WindowTrade)+"_Days_Close"]=StockData["close"].shift(-WindowTrade)
  StockData[str(WindowTrade)+"_Days_Return"]=(1-StockData["Next_"+str(WindowTrade)+"_Days_Close"]/StockData["close"])
  StockData["Stochastic_Oscilator"]=StochasticOscilatorArray(StockData["close"],10)
  StockData["Stochastic_Oscilator_Volatility"]=StochasticOscilatorArray(StockData["Hist_Volatility_"+str(WindowVol)+"_Days"],10)
  StockData["Exponential_Moving_Average_Close"]=ExpMovingAverageArray(StockData["close"],22)
  StockData["Exponential_Moving_Average_Volatility"]=ExpMovingAverageArray(StockData["Hist_Volatility_"+str(WindowVol)+"_Days"],22)
  #StockData["MACD"]=MACD_Array(StockData["close"], 26, 12,9)
  StockData["Stochastic_Oscilator_Volume"]=StochasticOscilatorArray(StockData["volume"],10)
  
  
  #StockData["1st_closeprice_derivative"]=np.diff(StockData["close"])
  StockData["Diff_Trend"]="Undefined"
  StockData["1st_closeprice_derivative"]=0.000
  
  cont=1
  while cont<len(StockData["1st_closeprice_derivative"]):
    StockData["1st_closeprice_derivative"][cont]=float(float(StockData["close"][cont])-float(StockData["close"][cont-1]))  
    cont=cont+1
  
 
  StockData["2nd_closeprice_derivative"]=0.000
  cont=2
  while cont<len(StockData["2nd_closeprice_derivative"]):
    StockData["2nd_closeprice_derivative"][cont]=float(float(StockData["1st_closeprice_derivative"][cont])-float(StockData["1st_closeprice_derivative"][cont-1]))       
    cont=cont+1
  cont=1
  while cont<len(StockData["1st_closeprice_derivative"]):  
    if StockData["1st_closeprice_derivative"][cont] >0:
      StockData["Diff_Trend"][cont]="Upward"
    else:
      StockData["Diff_Trend"][cont]="Downward"
    cont=cont+1
  
  StockData["Diff_Momentum"]="Undefined"
  cont=2
  while cont<len(StockData["2nd_closeprice_derivative"]):
    if float(float(StockData["1st_closeprice_derivative"][cont])*float(StockData["2nd_closeprice_derivative"][cont]))>0:
      StockData["Diff_Momentum"][cont]="Increasing"
    else:
      StockData["Diff_Momentum"][cont]="Decreasing"
    cont=cont+1  
  StockData["Diff_Trend_EMA"]="Undefined"
  StockData["1st_EMA_derivative"]=0.000
  
  cont=1
  while cont<len(StockData["1st_EMA_derivative"]):
    StockData["1st_EMA_derivative"][cont]=float(float(StockData["Exponential_Moving_Average_Close"][cont])-float(StockData["Exponential_Moving_Average_Close"][cont-1]))  
    cont=cont+1
  
 
  StockData["2nd_EMA_derivative"]=0.000
  cont=2
  while cont<len(StockData["2nd_EMA_derivative"]):
    StockData["2nd_EMA_derivative"][cont]=float(float(StockData["1st_EMA_derivative"][cont])-float(StockData["1st_closeprice_derivative"][cont-1]))       
    cont=cont+1
  

  #StockData.loc[StockData["1st_closeprice_derivative"]>0,"Diff_Trend"]="Upward"
  #StockData.loc[StockData["1st_closeprice_derivative"]>0,"Diff_Trend"]="Downward"
  cont=1
  while cont<len(StockData["1st_EMA_derivative"]):  
    if StockData["1st_EMA_derivative"][cont] >0:
      StockData["Diff_Trend_EMA"][cont]="Upward"
    else:
      StockData["Diff_Trend_EMA"][cont]="Downward"
    cont=cont+1
  
  StockData["Diff_Momentum_EMA"]="Undefined"
  cont=2
  while cont<len(StockData["2nd_EMA_derivative"]):
    if float(float(StockData["1st_EMA_derivative"][cont])*float(StockData["2nd_EMA_derivative"][cont]))>0:
      StockData["Diff_Momentum_EMA"][cont]="Increasing"
    else:
      StockData["Diff_Momentum_EMA"][cont]="Decreasing"
    cont=cont+1
  
  StockData["Stochastic_Overbought"]= "Average"
  StockData.loc[StockData["Stochastic_Oscilator"]>=0.8,"Stochastic_Overbought"]="Overbought"
  StockData.loc[StockData["Stochastic_Oscilator"]<=0.2,"Stochastic_Overbought"]="Oversold"
  
  StockData["Stochastic_Overbought_Volatility"]= "Average"
  StockData.loc[StockData["Stochastic_Oscilator_Volatility"]>=0.8,"Stochastic_Overbought_Volatility"]="Overbought"
  StockData.loc[StockData["Stochastic_Oscilator_Volatility"]<=0.2,"Stochastic_Overbought_Volatility"]="Oversold"

  StockData["Stochastic_Overbought_Volume"]= "Average"
  StockData.loc[StockData["Stochastic_Oscilator_Volume"]>=0.8,"Stochastic_Overbought_Volume"]="Overbought"
  StockData.loc[StockData["Stochastic_Oscilator_Volume"]<=0.2,"Stochastic_Overbought_Volume"]="Oversold"
 

  return StockData

def FilterDataFrame(dataframe, Column, Criteria,Condition="="):
  #filtra do dataframe pela condicao solicitada'
  FilteredDataFrame=pd.DataFrame() 
  if Condition == "=":
    ConditionTrue=dataframe[Column]==Criteria
  if Condition == "=<" or Condition == "<=":
    ConditionTrue=dataframe[Column]>=Criteria
  if Condition == "=>" or Condition == ">=":
    ConditionTrue=dataframe[Column]<=Criteria
  FilteredDataFrame=dataframe[ConditionTrue]
  FilteredDataFrame.index = pd.RangeIndex(len(FilteredDataFrame.index))
  return FilteredDataFrame






Contem funcoes estatisticas como o teste de Kolmogorv-smirnov (Kolmogorov_Smirnov_Test()) e calculo do intervalo de confian;a para media: mean_confidence_interval()

In [0]:
#@title
# statistics: este código contem análises estatísticas bayesianas do código

def Kolmogorov_Smirnov_Test(Sample1, Sample2, Significance=0.05, verbose=False):
  #Recebe duas amostras, calcula a estatistica de Kolmogorov-Smirnov e retorna 
  #os histogramas, medias, desvios padrao e se as amostras tem ou nao a mesma
  #distribuicao para a significancia desejada
  
  #Remove nans:
  Sample1=Sample1[~np.isnan(Sample1)]
  Sample2=Sample2[~np.isnan(Sample2)]
  
  result=list(ks_2samp(Sample1,Sample2))
  mystring=[]
  if result[1]>=Significance:
    if verbose==True: print("H0 not Rejected: Samples might have the same distribution")
    mystring.append(False)
  else:
    if verbose==True:print("H0 Rejected: Samples does not have the same distribution")
    mystring.append(True)
  if verbose==True:
    print("Sample 1 Statistics: ")
    print("Sample 1 average: "+str(np.mean(Sample1)))
    print("Sample 1 Std: "+str(np.std(Sample1)))
    print("Sample 2 Statistics: ")
    print("Sample 2 average: "+str(np.mean(Sample2)))
    print("Sample 2 Std: "+str(np.std(Sample2)))

    #Estimation of probability distribution using gaussian kernels
    KDEpdf_Samp1=gaussian_kde(Sample1)
    KDEpdf_Samp2=gaussian_kde(Sample2)
    import matplotlib.pyplot as plt
    plt.style.use("ggplot")
    x=np.linspace(-0.2,0.2,1500)
    plt.plot(x,KDEpdf_Samp1(x),'r',label="KDE Estimation Sample 1",color="blue")
    plt.hist(Sample1,normed=1,color="cyan",alpha=0.8)     
    plt.legend()
    plt.title("Parametric distribution of sample 1 x original histogram")
    plt.show()

    #pode-se usar o rv_histogram do scipy.stats tambem:
    #hist=np.histogram(data,bins=100)
    #hist_distributionscipy.stats.rv_histogram(hist)
    plt.plot(x,KDEpdf_Samp2(x),'r',label="KDE Estimation Sample 2",color="blue")
    plt.hist(Sample2,normed=1,color="cyan",alpha=0.8)     
    plt.legend()
    plt.title("Parametric distribution of sample 2 x original histogram")
    plt.show()
  mystring.append(result[1])   # p-value do teste de Kolmogorov-Smirnov          
  mystring.append(np.mean(Sample1)) #média amostra 1
  mystring.append(np.std(Sample1)) #desvio padrao amostra 1 
  mystring.append(np.mean(Sample2))#média amostra 2 
  mystring.append(np.std(Sample2)) #desvio padrao amostra 2           
  return mystring
              
              
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h
              




In [0]:
#@title
#Options Payoff Calculation: este trecho contém funções que retornam payoff de opções e suas estratégias:


def putpayoff(BS,premium,Strike,ClosePrice,Quantity):
  premium=math.fabs(premium)
  if BS=='BUY':
    if Strike >= ClosePrice:
      payoff=((Strike-ClosePrice)-premium)*Quantity
    else:
      payoff=-premium*Quantity
  elif BS=='SELL':
    if Strike >= ClosePrice:
      payoff=((ClosePrice-Strike)+premium)*Quantity
    else:
      payoff=premium*Quantity
  return payoff

def callpayoff(BS,premium,Strike,ClosePrice,Quantity):
  premium=math.fabs(premium)
  if BS=='BUY':
    if ClosePrice >= Strike:
      payoff=((ClosePrice-Strike)-premium)*Quantity
    else:
      payoff=-premium*Quantity
  elif BS=='SELL':
    if ClosePrice >= Strike:
      payoff=((Strike-ClosePrice)+premium)*Quantity
    else:
      payoff=premium*Quantity
  return payoff

def Boipayoff(lowcallpremium,highcallpremium,lowcallStrike,highcallStrike,ClosePrice,Quantity,highlowratio=3):
  #venda de uma opção e compra de maior quantidade de opção de strikesuperior - Compra de volatilidade
  lowcallpayoff=callpayoff('SELL',lowcallpremium,lowcallStrike,ClosePrice,Quantity)
  highcallpayoff=callpayoff('BUY',highcallpremium,highcallStrike,ClosePrice,highlowratio*Quantity)
  payoff=highcallpayoff+lowcallpayoff
  return payoff

def IronBoipayoff(lowcallpremium,highcallpremium,lowcallStrike,highcallStrike,ClosePrice,Quantity,highlowratio=3):
  #venda de uma opção e compra de maior quantidade de opção de strikesuperior - Compra de volatilidade
  lowcallpayoff=putpayoff('BUY',lowcallpremium,lowcallStrike,ClosePrice,Quantity)
  highcallpayoff=putpayoff('SELL',highcallpremium,highcallStrike,ClosePrice,highlowratio*Quantity)
  payoff=highcallpayoff+lowcallpayoff
  return payoff

def TravaAltapayoff(CP,premiumStrikeBaixo, premiumStrikeAlto, StrikeBaixo, StrikeAlto, Quantity,ClosePrice):
  if CP=='CALL':
    lowpayoff=callpayoff('BUY',premiumStrikeBaixo, StrikeBaixo,ClosePrice, Quantity)
    highpayoff=callpayoff('SELL',premiumStrikeAlto, StrikeAlto,ClosePrice, Quantity)
  elif CP=='PUT':
    lowpayoff=putpayoff('SELL',premiumStrikeAlto, StrikeAlto,ClosePrice, Quantity)
    highpayoff=putpayoff('BUY',premiumStrikeBaixo, StrikeBaixo,ClosePrice, Quantity)
  payoff=highpayoff+lowpayoff
  return payoff

def TravaBaixapayoff(CP,premiumStrikeBaixo, premiumStrikeAlto, StrikeBaixo, StrikeAlto, Quantity,ClosePrice):
  if CP=='CALL':
    lowpayoff=callpayoff('SELL',premiumStrikeBaixo, StrikeBaixo,ClosePrice, Quantity)
    highpayoff=callpayoff('BUY',premiumStrikeAlto, StrikeAlto,ClosePrice, Quantity)
  elif CP=='PUT':
    lowpayoff=putpayoff('BUY',premiumStrikeAlto, StrikeAlto,ClosePrice, Quantity)
    highpayoff=putpayoff('SELL',premiumStrikeBaixo, StrikeBaixo,ClosePrice, Quantity)
  payoff=highpayoff+lowpayoff
  return payoff

def Vacapayoff(lowcallpremium,highcallpremium,lowcallStrike,highcallStrike,ClosePrice,Quantity,lockpremium,lockstrike,highlowratio=3):
  # venda de volatilidade: compra de opção, venda de maior quantidade de strike superior, trava em strike mais alto ainda:
  lowcallpayoff=callpayoff('BUY',lowcallpremium,lowcallStrike,ClosePrice,Quantity)
  highcallpayoff=callpayoff('SELL',highcallpremium,highcallStrike,ClosePrice,highlowratio*Quantity)
  lockcallpayoff=callpayoff('BUY',lockpremium,lockstrike,ClosePrice,Quantity*(highlowratio-1))
  payoff=highcallpayoff+lowcallpayoff+lockcallpayoff
  return payoff

def VacaRevertidapayoff(lowcallpremium,highcallpremium,lowcallStrike,highcallStrike,ClosePrice,Quantity,lockpremium,lockstrike,highlowratio=5):
  # venda de volatilidade, mas ao contrário da vaca, volta a ganhar numa explosão de vol
  lowcallpayoff=callpayoff('BUY',lowcallpremium,lowcallStrike,ClosePrice,Quantity)
  highcallpayoff=callpayoff('SELL',highcallpremium,highcallStrike,ClosePrice,highlowratio*Quantity)
  lockcallpayoff=callpayoff('BUY',lockpremium,lockstrike,ClosePrice,highlowratio*Quantity)
  payoff=highcallpayoff+lowcallpayoff+lockcallpayoff
  return payoff

def Condor(IRON,lowoptpremium,highoptpremium,lowlockpremium,highlockpremium,lowoptStrike,highoptStrike,lowlockstrike,highlockstrike,ClosePrice,Quantity):
  #venda de volatilidade: se deseja que o preço permaneça em uma faixa
  if IRON==True:
    lowlockpayoff=putpayoff('BUY',lowlockpremium,lowlockstrike,ClosePrice,Quantity)
    lowoptpayoff=putpayoff('SELL',lowoptpremium,lowoptStrike,ClosePrice,Quantity)
    highoptpayoff=callpayoff('SELL',highoptpremium,highoptStrike,ClosePrice,Quantity)
    highlockpayoff=callpayoff('BUY',highlockpremium,highlockstrike,ClosePrice,Quantity)
  else:
    lowlockpayoff=callpayoff('BUY',lowlockpremium,lowlockstrike,ClosePrice,Quantity)
    lowoptpayoff=callpayoff('SELL',lowoptpremium,lowoptStrike,ClosePrice,Quantity)
    highoptpayoff=callpayoff('SELL',highoptpremium,highoptStrike,ClosePrice,Quantity)
    highlockpayoff=callpayoff('BUY',highlockpremium,highlockstrike,ClosePrice,Quantity)
  payoff=highlockpayoff+highoptpayoff+lowoptpayoff+lowlockpayoff
  return payoff

In [0]:
#@title
#Obtencao dos dados

OptionChain=getOptionChain(Asset, fromDate, toDate)
#Ha bug nas duas linhas de codigo abaixo - havera erro se for rodado antes da abertura do mercado, 
#necessario buscar dados de mais dias para mitigar este erro
DfSpot=getIntradayData(Asset,date.today().strftime('%Y-%m-%d'))

SpotPrice=DfSpot['close'].iloc[-1]
if SpotPrice == 0: 
  DfSpot=getIntradayData(Asset,(date.today() - datetime.timedelta(1)).strftime('%Y-%m-%d'))
  SpotPrice=DfSpot['close'].iloc[-1]
                                          
                                          
#filtraSpotPricer somente o strike desejado:
OptionChain=FilterDataFrame(OptionChain,"expirationDate",date.today(),Condition='<=')


OptionChain=FilterDataFrame(OptionChain,"expirationDate",date.today()+ datetime.timedelta(days=1),Condition='<=')
#ordenar o dataframe
OptionChain.sort_values(by=['expirationDate','putCall','strikePrice'],ascending=False)

OptionChainPut=FilterDataFrame(OptionChain,"putCall",'PUT')
OptionChainCall=FilterDataFrame(OptionChain,"putCall",'CALL')

 
#Obter strikes ATM:
cont=1
while cont<len(OptionChainCall["strikePrice"]) or cont<len(OptionChainPut["strikePrice"]):
    if cont<len(OptionChainCall["strikePrice"]):
      if OptionChainCall["strikePrice"][cont]>SpotPrice and OptionChainCall["strikePrice"][cont-1]<SpotPrice:
        ATMStrikeCall=OptionChainCall["strikePrice"][cont]
        ATMStrikeCallPOS=cont

    if cont<len(OptionChainPut["strikePrice"]):
      
      if OptionChainPut["strikePrice"][cont]>SpotPrice and OptionChainPut["strikePrice"][cont-1]<SpotPrice:
        ATMStrikePut=OptionChainPut["strikePrice"][cont]
        ATMStrikePutPOS=cont
    cont=cont+1

    
#OptionStrikePrices=[]
#Obter os preços das opções call e put no momento
#ListColumns(OptionChain) #avaliar o que vem
#OptionChain.head(5)
#print(OptionChain.dtypes)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:128: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.


In [0]:
#@title
#OptionChain.head(5)

In [0]:
#@title
StockData=FullStockDataFrame(StartDate, EndDate, Asset, WindowVol, WindowTrade)

In [0]:
#@title
#estratégia: usar as condições do último dia de negociação para avaliar probabilidades de payoff do último dia

#Avaliação por momento de preço e volume
StochasticOverbought=StockData.loc[StockData.index[-1], "Stochastic_Overbought"]
VolumeOverbough=StockData.loc[StockData.index[-1], "Stochastic_Overbought_Volume"]
VolatilityOverbought=StockData.loc[StockData.index[-1], "Stochastic_Overbought_Volatility"]
SpotClosePrice=StockData.loc[StockData.index[-1],"close"]
StockDataInter=pd.DataFrame()
StockDataInterTest=FilterDataFrame(StockData,"Stochastic_Overbought_Volume",VolumeOverbough)
if len(StockDataInter)<5:
  StockDataInter=StockData
else:
  StockDataInter=StockDataInterTest
  
StockDataInterTest=FilterDataFrame(StockDataInter,"Stochastic_Overbought_Volatility",VolatilityOverbought)
if len(StockDataInter)<5:
  StockDataInter=StockDataInter
else:
  StockDataInter=StockDataInterTest

StockDataInterTest=FilterDataFrame(StockDataInter,"Stochastic_Overbought",StochasticOverbought)
if len(StockDataInter)<5:
  StockDataInter=StockDataInter
else:
  StockDataInter=StockDataInterTest

#estimativa da função densidade de probabilidade para as condições do mercado em questão:
DataForPriceEstimator=pd.DataFrame()
DataForPriceEstimator["Prices_given_return"]=(1+StockDataInter[str(WindowTrade)+"_Days_Return"])*SpotClosePrice

DataForPriceEstimator.dropna(how='any', inplace=True)

DataForPriceEstimator=DataForPriceEstimator.sort_values(by="Prices_given_return",ascending=False)
Kernel=gaussian_kde(DataForPriceEstimator["Prices_given_return"])
DataForPriceEstimator["Price_PDF"]=Kernel.evaluate(DataForPriceEstimator["Prices_given_return"]) #Obter PDF para cada ponto

#Testes das estratégias:
cont=1
DataForPriceEstimator['TravaAltapayoffCALL']=0.0
DataForPriceEstimator['TravaBaixapayoffCALL']=0.0
DataForPriceEstimator['BoiATMpayoffCALL']=0.0
DataForPriceEstimator['CondorpayoffCALL']=0.0
DataForPriceEstimator['IronBoiATMpayoffCALL']=0.0

while cont< len(DataForPriceEstimator["Prices_given_return"]):
  #Trava de Alta:
  #Trava de Alta ITM Call:
  premiumStrikeBaixo=OptionChainCall['ask'][ATMStrikeCallPOS]
  premiumStrikeAlto=OptionChainCall['bid'][ATMStrikeCallPOS+2]
  StrikeBaixo=OptionChainCall['strikePrice'][ATMStrikeCallPOS]
  StrikeAlto=OptionChainCall['strikePrice'][ATMStrikeCallPOS+2]
  DataForPriceEstimator['TravaAltapayoffCALL'][cont]= float(TravaAltapayoff('CALL',premiumStrikeBaixo, premiumStrikeAlto, StrikeBaixo, StrikeAlto, Quantity,DataForPriceEstimator["Prices_given_return"][cont]))
  #lowpayoff=callpayoff('BUY',premiumStrikeBaixo, StrikeBaixo,DataForPriceEstimator["Prices_given_return"][cont], Quantity)
  #highpayoff=callpayoff('SELL',premiumStrikeAlto, StrikeAlto,DataForPriceEstimator["Prices_given_return"][cont], Quantity)

  
  #Trava de Baixa:
  #Trava de Baixa ATM Call:
  premiumStrikeBaixo=OptionChainCall['bid'][ATMStrikeCallPOS]
  premiumStrikeAlto=OptionChainCall['ask'][ATMStrikeCallPOS+2]
  StrikeBaixo=OptionChainCall['strikePrice'][ATMStrikeCallPOS]
  StrikeAlto=OptionChainCall['strikePrice'][ATMStrikeCallPOS+2]
  DataForPriceEstimator['TravaBaixapayoffCALL'][cont]= float(TravaBaixapayoff('CALL',premiumStrikeBaixo, premiumStrikeAlto, StrikeBaixo, StrikeAlto, Quantity,DataForPriceEstimator["Prices_given_return"][cont]))

  #Compra de Volatilidade:
  #Boi ATM:
  premiumStrikeBaixo=OptionChainCall['bid'][ATMStrikeCallPOS]
  premiumStrikeAlto=OptionChainCall['ask'][ATMStrikeCallPOS+2]
  StrikeBaixo=OptionChainCall['strikePrice'][ATMStrikeCallPOS]
  StrikeAlto=OptionChainCall['strikePrice'][ATMStrikeCallPOS+2]
  DataForPriceEstimator['BoiATMpayoffCALL'][cont]= float(Boipayoff(premiumStrikeBaixo,premiumStrikeAlto,StrikeBaixo,StrikeAlto,DataForPriceEstimator["Prices_given_return"][cont],Quantity,highlowratio=2))
  
  
  #Iron Boi ATM:
  premiumStrikeBaixo=OptionChainCall['bid'][ATMStrikePutPOS]
  premiumStrikeAlto=OptionChainCall['ask'][ATMStrikePutPOS-2]
  StrikeBaixo=OptionChainCall['strikePrice'][ATMStrikePutPOS]
  StrikeAlto=OptionChainCall['strikePrice'][ATMStrikePutPOS-2]
  DataForPriceEstimator['IronBoiATMpayoffCALL'][cont]= float(IronBoipayoff(premiumStrikeBaixo,premiumStrikeAlto,StrikeBaixo,StrikeAlto,DataForPriceEstimator["Prices_given_return"][cont],Quantity,highlowratio=2))
  
  #Venda de Volatilidade:
  #Condor:  
  premiumTravaBaixa=OptionChainCall['bid'][ATMStrikeCallPOS-2]             
  premiumStrikeBaixo=OptionChainCall['ask'][ATMStrikeCallPOS-1]
  premiumStrikeAlto=OptionChainCall['ask'][ATMStrikeCallPOS+1]
  premiumTravaAlta=OptionChainCall['bid'][ATMStrikeCallPOS+2]
  StrikeBaixoTrava=OptionChainCall['strikePrice'][ATMStrikeCallPOS-2]           
  StrikeBaixo=OptionChainCall['strikePrice'][ATMStrikeCallPOS-1]
  StrikeAlto=OptionChainCall['strikePrice'][ATMStrikeCallPOS+1]
  StrikeAltoTrava=OptionChainCall['strikePrice'][ATMStrikeCallPOS+2]
  DataForPriceEstimator['CondorpayoffCALL'][cont]=float(Condor(False,premiumStrikeBaixo,premiumStrikeAlto,premiumTravaBaixa,premiumTravaAlta,StrikeBaixo,StrikeAlto,StrikeBaixoTrava,StrikeAltoTrava,DataForPriceEstimator["Prices_given_return"][cont],Quantity))              
  cont=cont+1

#determinação da operação de melhor payoff:
                
MelhorOperação="Nenhuma"
MelhorPayoff=0
                
#payoff TravaAlta:
PayoffTeste=trapz(DataForPriceEstimator['TravaAltapayoffCALL'], DataForPriceEstimator["Price_PDF"])
if PayoffTeste>MelhorPayoff:
   MelhorPayoff=PayoffTeste
   MelhorOperação="TravaAlta"
                
#payoff TravaBaixa:
PayoffTeste=trapz(DataForPriceEstimator['TravaBaixapayoffCALL'], DataForPriceEstimator["Price_PDF"])
if PayoffTeste>MelhorPayoff:
   MelhorPayoff=PayoffTeste
   MelhorOperação="TravaBaixa"    
                
#payoff Boi:
PayoffTeste=trapz(DataForPriceEstimator['BoiATMpayoffCALL'], DataForPriceEstimator["Price_PDF"])
if PayoffTeste>MelhorPayoff:
   MelhorPayoff=PayoffTeste
   MelhorOperação="Boi"    

#Payoff Iron Boi   
PayoffTeste=trapz(DataForPriceEstimator['IronBoiATMpayoffCALL'], DataForPriceEstimator["Price_PDF"])
if PayoffTeste>MelhorPayoff:
   MelhorPayoff=PayoffTeste
   MelhorOperação="Iron Boi"   


#payoff Condor:
PayoffTeste=trapz(DataForPriceEstimator['CondorpayoffCALL'], DataForPriceEstimator["Price_PDF"])
if PayoffTeste>MelhorPayoff:
   MelhorPayoff=PayoffTeste
   MelhorOperação="Condor"   

In [0]:
#@title
#Output Verboso:
#payoff TravaAlta:
PayoffTeste=trapz(DataForPriceEstimator['TravaAltapayoffCALL'], DataForPriceEstimator["Price_PDF"])
print("TravaAlta:")
print("Strike Baixo: " + str(StrikeBaixo))
print("Strike Alto: " + str(StrikeAlto))
print("Payoff Esperado: "+ str(PayoffTeste))
Gamma=OptionChainCall['gamma'][ATMStrikeCallPOS]-OptionChainCall['gamma'][ATMStrikeCallPOS+2]
Delta=OptionChainCall['delta'][ATMStrikeCallPOS]-OptionChainCall['delta'][ATMStrikeCallPOS+2]
Theta=OptionChainCall['theta'][ATMStrikeCallPOS]-OptionChainCall['theta'][ATMStrikeCallPOS+2]
VE=-premiumStrikeBaixo+premiumStrikeAlto
print("Gamma:" + str(round(Gamma,4)) + " Delta:" + str(round(Delta,4)) +" Theta:" + str(round(Theta,4)) +" Valor Extrínseco: " + str(round(VE,4)))
if OptionChainCall['gamma'][ATMStrikeCallPOS+2]+OptionChainCall['delta'][ATMStrikeCallPOS+2] > premiumStrikeAlto:
  print("Alerta Opção NV!")
else:
  print("Teste NV OK")
print("")
  
#payoff TravaBaixa:
PayoffTeste=trapz(DataForPriceEstimator['TravaBaixapayoffCALL'], DataForPriceEstimator["Price_PDF"])
print("TravaBaixa:")
print("Strike Baixo: " + str(StrikeBaixo))
print("Strike Alto: " + str(StrikeAlto))
print("Payoff Esperado: "+ str(PayoffTeste))  
Gamma=-OptionChainCall['gamma'][ATMStrikeCallPOS]+OptionChainCall['gamma'][ATMStrikeCallPOS+2]
Delta=-OptionChainCall['delta'][ATMStrikeCallPOS]+OptionChainCall['delta'][ATMStrikeCallPOS+2]
Theta=-OptionChainCall['theta'][ATMStrikeCallPOS]+OptionChainCall['theta'][ATMStrikeCallPOS+2]
VE=+premiumStrikeBaixo-premiumStrikeAlto
print("Gamma:" + str(round(Gamma,4)) + " Delta:" + str(round(Delta,4)) +" Theta:" + str(round(Theta,4)) +" Valor Extrínseco: " + str(round(VE,4)))
if OptionChainCall['gamma'][ATMStrikeCallPOS]+OptionChainCall['delta'][ATMStrikeCallPOS] > premiumStrikeBaixo:
  print("Alerta Opção NV!")
else:
  print("Teste NV OK")
print("")

#payoff Boi:
PayoffTeste=trapz(DataForPriceEstimator['BoiATMpayoffCALL'], DataForPriceEstimator["Price_PDF"])
print("Boi:")
print("Strike Baixo: " + str(StrikeBaixo))
print("Strike Alto: " + str(StrikeAlto))
print("Payoff Esperado: "+ str(PayoffTeste))
Gamma=-OptionChainCall['gamma'][ATMStrikeCallPOS]+2*OptionChainCall['gamma'][ATMStrikeCallPOS+2]
Delta=-OptionChainCall['delta'][ATMStrikeCallPOS]+2*OptionChainCall['delta'][ATMStrikeCallPOS+2]
Theta=-OptionChainCall['theta'][ATMStrikeCallPOS]+2*OptionChainCall['theta'][ATMStrikeCallPOS+2]
VE=premiumStrikeBaixo-2*premiumStrikeAlto
print("Gamma:" + str(round(Gamma,4)) + " Delta:" + str(round(Delta,4)) +" Theta:" + str(round(Theta,4)) +" Valor Extrínseco: " + str(round(VE,4)))
if OptionChainCall['gamma'][ATMStrikeCallPOS]+OptionChainCall['delta'][ATMStrikeCallPOS] > premiumStrikeBaixo:
  print("Alerta Opção NV!")
else:
  print("Teste NV OK")
print("")

#payoff Iron Boi:
PayoffTeste=trapz(DataForPriceEstimator['IronBoiATMpayoffCALL'], DataForPriceEstimator["Price_PDF"])
print("iron Boi:")
print("Strike Baixo: " + str(OptionChainCall['strikePrice'][ATMStrikePutPOS]))
print("Strike Alto: " + str(OptionChainCall['strikePrice'][ATMStrikePutPOS-2]))
print("Payoff Esperado: "+ str(PayoffTeste))
Gamma=-OptionChainCall['gamma'][ATMStrikePutPOS]+2*OptionChainCall['gamma'][ATMStrikePutPOS-2]
Delta=-OptionChainCall['delta'][ATMStrikePutPOS]+2*OptionChainCall['delta'][ATMStrikePutPOS-2]
Theta=-OptionChainCall['theta'][ATMStrikePutPOS]+2*OptionChainCall['theta'][ATMStrikePutPOS-2]
VE=premiumStrikeBaixo-2*premiumStrikeAlto
print("Gamma:" + str(round(Gamma,4)) + " Delta:" + str(round(Delta,4)) +" Theta:" + str(round(Theta,4)) +" Valor Extrínseco: " + str(round(VE,4)))
if OptionChainCall['gamma'][ATMStrikePutPOS]+OptionChainCall['delta'][ATMStrikePutPOS] > premiumStrikeBaixo:
  print("Alerta Opção NV!")
else:
  print("Teste NV OK")
print("")
  
#payoff Condor:
PayoffTeste=trapz(DataForPriceEstimator['CondorpayoffCALL'], DataForPriceEstimator["Price_PDF"])
print("Condor:")
print("Trava Baixa: " + str(StrikeBaixoTrava))
print("Strike Baixo: " + str(StrikeBaixo))
print("Strike Alto: " + str(StrikeAlto))
print("Trava Alta: " + str(StrikeAltoTrava))
print("Payoff Esperado: "+ str(PayoffTeste))
Gamma=-OptionChainCall['gamma'][ATMStrikeCallPOS-1]-OptionChainCall['gamma'][ATMStrikeCallPOS+1]+OptionChainCall['gamma'][ATMStrikeCallPOS-2]+OptionChainCall['gamma'][ATMStrikeCallPOS+2]
Delta=-OptionChainCall['delta'][ATMStrikeCallPOS-1]-OptionChainCall['delta'][ATMStrikeCallPOS+1]+OptionChainCall['delta'][ATMStrikeCallPOS-2]+OptionChainCall['delta'][ATMStrikeCallPOS+2]
Theta=-OptionChainCall['theta'][ATMStrikeCallPOS-1]-OptionChainCall['theta'][ATMStrikeCallPOS+1]+OptionChainCall['theta'][ATMStrikeCallPOS-2]+OptionChainCall['theta'][ATMStrikeCallPOS+2]
VE=+premiumStrikeBaixo-premiumStrikeAlto
print("Gamma:" + str(round(Gamma,4)) + " Delta:" + str(round(Delta,4)) +" Theta:" + str(round(Theta,4)) +" Valor Extrínseco: " + str(round(VE,4)))
if OptionChainCall['gamma'][ATMStrikeCallPOS-1]+OptionChainCall['gamma'][ATMStrikeCallPOS+1]+OptionChainCall['delta'][ATMStrikeCallPOS+1]+OptionChainCall['delta'][ATMStrikeCallPOS-1] > premiumStrikeBaixo+premiumStrikeAlto:
  print("Alerta Opção NV!")
else:
  print("Teste NV OK")
print("")

TravaAlta:
Strike Baixo: 290.0
Strike Alto: 291.0
Payoff Esperado: 0.018212638610850915
Gamma:-0.002 Delta:0.045 Theta:-0.029 Valor Extrínseco: -0.67
Teste NV OK

TravaBaixa:
Strike Baixo: 290.0
Strike Alto: 291.0
Payoff Esperado: -0.017795444111640482
Gamma:0.002 Delta:-0.045 Theta:0.029 Valor Extrínseco: 0.67
Teste NV OK

Boi:
Strike Baixo: 290.0
Strike Alto: 291.0
Payoff Esperado: 0.0016041001016444678
Gamma:-0.124 Delta:0.372 Theta:-0.337 Valor Extrínseco: -2.44
Teste NV OK

Condor:
Trava Baixa: 289.5
Strike Baixo: 290.0
Strike Alto: 291.0
Trava Alta: 291.5
Payoff Esperado: 0.010028776031707268
Gamma:0.0 Delta:-0.003 Theta:0.004 Valor Extrínseco: 0.67
Teste NV OK



In [0]:
#@title
#output usuário:

print("Melhor Operação: "+ MelhorOperação)
if MelhorOperação=="TravaAlta":
  print("Strike Baixo: " + str(StrikeBaixo))
  print("Strike Alto: " + str(StrikeAlto))
  print("Payoff Esperado: "+ str(MelhorPayoff))
elif MelhorOperação=="TravaBaixa":
  print("Strike Baixo: " + str(StrikeBaixo))
  print("Strike Alto: " + str(StrikeAlto))
  print("Payoff Esperado: "+ str(MelhorPayoff))
elif MelhorOperação=="Boi":
  print("Strike Baixo: " + str(StrikeBaixo))
  print("Strike Alto: " + str(StrikeAlto))
  print("Payoff Esperado: "+ str(MelhorPayoff))
elif MelhorOperação=="Condor":
  print("Trava Baixa: " + str(StrikeBaixoTrava))
  print("Strike Baixo: " + str(StrikeBaixo))
  print("Strike Alto: " + str(StrikeAlto))
  print("Trava Alta: " + str(StrikeAltoTrava))
  print("Payoff Esperado: "+ str(MelhorPayoff))

Melhor Operação: TravaAlta
Strike Baixo: 290.0
Strike Alto: 291.0
Payoff Esperado: 0.018212638610850915


In [0]:
#@title
#DataForPriceEstimator